In [1]:
import sys
#sys.path.insert(0, "../..")

from typing import Callable, Protocol, Dict, Optional, Iterator, List, Tuple

import gin
from pathlib import Path
import numpy as np
import math
from datetime import datetime

import torch
from torch import nn
import torch.optim as optim
from torch.optim import Optimizer
from torchinfo import summary
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# ------------------------------------------------------------------------------- #
## Alle imports benodigd voor de functie train_loop uit train_model.py 

import tensorflow as tf  # noqa: F401

# needed to make summarywriter load without error
from loguru import logger
from numpy import Inf
from ray import tune


from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm

2022-12-25 07:59:30.074497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-25 07:59:37.802236: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-25 07:59:37.802468: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-25 07:59:37.802482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
## benodigde functie uitmake_dataset.py

@gin.configurable
def get_MNIST(  # noqa: N802
    data_dir: Path, batch_size: int
) -> Tuple[DataLoader, DataLoader]:

    training_data = datasets.FashionMNIST(
        root=data_dir,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root=data_dir,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

In [3]:
## Benodigde funties uit metrics.py

Tensor = torch.Tensor

class Metric:
    def __repr__(self) -> str:
        raise NotImplementedError

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        raise NotImplementedError

class Accuracy(Metric):
    def __repr__(self) -> str:
        return "Accuracy"

    def __call__(self, y: Tensor, yhat: Tensor) -> Tensor:
        return (yhat.argmax(dim=1) == y).sum() / len(yhat)

class GenericModel(Protocol):
    train: Callable
    eval: Callable
    parameters: Callable

    def __call__(self, *args, **kwargs) -> torch.Tensor:
        pass


def count_parameters(model: GenericModel) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


## uit data_tools.py
def dir_add_timestamp(log_dir: Optional[Path] = None) -> Path:
    if log_dir is None:
        log_dir = Path(".")
    log_dir = Path(log_dir)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M")
    log_dir = log_dir / timestamp
    logger.info(f"Logging to {log_dir}")
    if not log_dir.exists():
        log_dir.mkdir(parents=True)
    return log_dir

In [4]:
## Definieren functie train_loop

@gin.configurable
def trainloop(
    epochs: int,
    model: GenericModel,
    optimizer: torch.optim.Optimizer,
    learning_rate: float,
    loss_fn: Callable,
    metrics: List[Metric],
    train_dataloader: Iterator,
    test_dataloader: Iterator,
    log_dir: Path,
    train_steps: int,
    eval_steps: int,
    patience: int = 10,
    factor: float = 0.9,
    tunewriter: bool = False,
) -> GenericModel:
    
    optimizer_: torch.optim.Optimizer = optimizer(
        model.parameters(), lr=learning_rate
    )  # type: ignore

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer_,
        factor=factor,
        patience=patience,
    )

    if not tunewriter:
        log_dir = dir_add_timestamp(log_dir)
        writer = SummaryWriter(log_dir=log_dir)
        #write_gin(log_dir, gin.config_str())

        images, _ = next(iter(train_dataloader))
        if len(images.shape) == 4:
            grid = make_grid(images)
            writer.add_image("images", grid)
        writer.add_graph(model, images)

    for epoch in tqdm(range(epochs)):
        train_loss = trainbatches(
            model, train_dataloader, loss_fn, optimizer_, train_steps
        )

        metric_dict, test_loss = evalbatches(
            model, test_dataloader, loss_fn, metrics, eval_steps
        )

        scheduler.step(test_loss)

        if tunewriter:
            tune.report(
                iterations=epoch,
                train_loss=train_loss,
                test_loss=test_loss,
                **metric_dict,
            )
        else:
            writer.add_scalar("Loss/train", train_loss, epoch)
            writer.add_scalar("Loss/test", test_loss, epoch)
            for m in metric_dict:
                writer.add_scalar(f"metric/{m}", metric_dict[m], epoch)
            lr = [group["lr"] for group in optimizer_.param_groups][0]
            writer.add_scalar("learning_rate", lr, epoch)
            metric_scores = [f"{v:.4f}" for v in metric_dict.values()]
            logger.info(
                f"Epoch {epoch} train {train_loss:.4f} test {test_loss:.4f} metric {metric_scores}"  # noqa E501
            )

    return model

In [5]:
## uit train_model.py

def trainbatches(
    model: GenericModel,
    traindatastreamer: Iterator,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    train_steps: int,
) -> float:
    model.train()
    train_loss: float = 0.0
    for _ in tqdm(range(train_steps)):
        x, y = next(iter(traindatastreamer))
        optimizer.zero_grad()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.detach().numpy()
    train_loss /= train_steps
    return train_loss


## uit train_model.py
def evalbatches(
    model: GenericModel,
    testdatastreamer: Iterator,
    loss_fn: Callable,
    metrics: List[Metric],
    eval_steps: int,
) -> Tuple[Dict[str, float], float]:
    model.eval()
    test_loss: float = 0.0
    metric_dict: Dict[str, float] = {}
    for _ in range(eval_steps):
        x, y = next(iter(testdatastreamer))
        yhat = model(x)
        test_loss += loss_fn(yhat, y).detach().numpy()
        for m in metrics:
            metric_dict[str(m)] = (
                metric_dict.get(str(m), 0.0) + m(y, yhat).detach().numpy()
            )

    test_loss /= eval_steps
    for key in metric_dict:
        metric_dict[str(key)] = metric_dict[str(key)] / eval_steps
    return metric_dict, test_loss

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
## Model 1: CNN_een met een convolutional layer & drie linear layers

@gin.configurable
class CNN_1_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(6272, 3136),
            nn.ReLU(),
            nn.Linear(3136, 1568),
            nn.ReLU(),
            nn.Linear(1568, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [8]:
## Model 2.2: CNN_twee met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_2_2(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [9]:
## Model 2.2.16.32: CNN_twee met twee convolutional layers & twee linear layers & 16-32 filters

@gin.configurable
class CNN_2_2_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits



In [10]:
## Model 2.2.32.64: CNN_twee met twee convolutional layers & twee linear layers & 32-64 filters

@gin.configurable
class CNN_2_2_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits



In [11]:
## Model 2.2.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_2_2_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [12]:
## Model 2.2.128.256: CNN_twee met twee convolutional layers & twee linear layers & 128-256 filters

@gin.configurable
class CNN_2_2_128_256(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 4608),
            nn.ReLU(),
            nn.Linear(4608, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [13]:
## Model 2.3: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [14]:
## Model 2.3.16.32: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [15]:
## Model 2.3.32.64: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [16]:
## Model 2.3.64.128: CNN_twee met twee convolutional layers en drie linear layers

@gin.configurable
class CNN_2_3_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [17]:
## Model 2.4: CNN_twee met twee convolutional layers & vier linear layers

@gin.configurable
class CNN_2_4(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 144),
            nn.ReLU(),
            nn.Linear(144, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [18]:
## Model 2.4.16.32: CNN_twee met twee convolutional layers & vier linear layers

@gin.configurable
class CNN_2_4_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, 144),
            nn.ReLU(),
            nn.Linear(144, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [19]:
## Model 2.4.32.64: CNN_twee met twee convolutional layers & vier linear layers

@gin.configurable
class CNN_2_4_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [20]:
## Model 2.4.64.128: CNN_twee met twee convolutional layers & vier linear layers

@gin.configurable
class CNN_2_4_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [21]:
## Model 3.2: CNN_drie met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_3_2(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [22]:
## Model 3.2.16.32: CNN_drie met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_3_2_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [23]:
## Model 3.2.16.32: CNN_drie met twee convolutional layers & twee linear layers

@gin.configurable
class CNN_3_2_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [24]:
## Model 3.3: CNN_drie met drie convolutional layers

@gin.configurable
class CNN_3_3(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [25]:
## Model 3.3.16.32: CNN_drie met drie convolutional layers

@gin.configurable
class CNN_3_3_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(144, 72),
            nn.ReLU(),
            nn.Linear(72, 36),
            nn.ReLU(),
            nn.Linear(36, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [26]:
## Model 3.3.32.64: CNN_drie met drie convolutional layers

@gin.configurable
class CNN_3_3_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [27]:
## Model 3.4: CNN_drie met drie convolutional layers & vier linear layers

@gin.configurable
class CNN_3_4(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits


In [28]:
## Model 3.4.16.32: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_16_32(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [29]:
## Model 3.4.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_32_64(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, 288),
            nn.ReLU(),
            nn.Linear(288, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [30]:
## Model 3.4.64.128: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_64_128(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, 576),
            nn.ReLU(),
            nn.Linear(576, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [31]:
## Model 3.4.128.256: CNN_twee met twee convolutional layers & twee linear layers & 64-128 filters

@gin.configurable
class CNN_3_4_128_256(nn.Module):
    def __init__(
        self, num_classes: int, kernel_size: int, filter1: int, filter2: int
    ) -> None:
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, filter1, kernel_size=kernel_size, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filter1, filter2, kernel_size=kernel_size, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            )

        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(9216, 4608),
            nn.ReLU(),
            nn.Linear(4608, 2304),
            nn.ReLU(),
            nn.Linear(2304, 1152),
            nn.ReLU(),
            nn.Linear(1152, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

In [53]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

In [33]:
#print(summary(model))
#print(gin.config_str())

In [34]:
datadir =  Path("/home/admindme/code/ML22_opdracht1/data/raw/FashionMNIST")
train_dataloader, test_dataloader = get_MNIST(datadir, 64)
len(train_dataloader), len(test_dataloader)
accuracy = Accuracy()

In [38]:
## Stap 1 
model = CNN_1_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-25 08:00:27.647 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221225-0800
100%|██████████| 938/938 [03:56<00:00,  3.96it/s]
2022-12-25 08:04:29.908 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.9371 test 0.3217 metric ['0.8848']
100%|██████████| 938/938 [05:09<00:00,  3.03it/s]
2022-12-25 08:09:43.563 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.2739 test 0.3106 metric ['0.8923']
100%|██████████| 938/938 [05:09<00:00,  3.03it/s]
2022-12-25 08:14:57.762 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.2679 test 0.3897 metric ['0.8761']
100%|██████████| 938/938 [05:09<00:00,  3.03it/s]
2022-12-25 08:20:11.551 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.2545 test 0.3497 metric ['0.8896']
100%|██████████| 938/938 [05:10<00:00,  3.02it/s]
2022-12-25 08:25:26.293 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.2343 test 0.3511 metric ['0.8897']
100%|██████████| 5/5 [24:57<00:00, 299.56s/it]


In [ ]:
## Stap 1 
model = CNN_2_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 19:25:52.473 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-1925
100%|██████████| 938/938 [00:34<00:00, 26.83it/s]
2022-12-23 19:26:30.446 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.5847 test 0.4695 metric ['0.8375']
100%|██████████| 938/938 [00:36<00:00, 25.44it/s]
2022-12-23 19:27:09.411 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4138 test 0.4384 metric ['0.8357']
100%|██████████| 938/938 [00:36<00:00, 25.44it/s]
2022-12-23 19:27:48.474 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3785 test 0.3789 metric ['0.8573']
100%|██████████| 938/938 [00:36<00:00, 25.61it/s]
2022-12-23 19:28:27.240 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3652 test 0.3838 metric ['0.8612']
100%|██████████| 938/938 [00:36<00:00, 25.95it/s]
2022-12-23 19:29:05.451 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3380 test 0.3648 metric ['0.8704']
100%|██████████| 5/5 [03:12<00:00, 38.43s/it]


In [ ]:
## Stap 1 
model = CNN_3_3().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 20:20:09.950 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2020
100%|██████████| 938/938 [00:29<00:00, 32.08it/s]
2022-12-23 20:20:41.517 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.5654 test 0.4180 metric ['0.8495']
100%|██████████| 938/938 [00:29<00:00, 31.50it/s]
2022-12-23 20:21:13.407 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.3716 test 0.4064 metric ['0.8465']
100%|██████████| 938/938 [00:30<00:00, 30.92it/s]
2022-12-23 20:21:45.898 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3417 test 0.3641 metric ['0.8685']
100%|██████████| 938/938 [00:24<00:00, 38.41it/s]
2022-12-23 20:22:12.420 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3225 test 0.3725 metric ['0.8714']
100%|██████████| 938/938 [00:24<00:00, 37.94it/s]
2022-12-23 20:22:39.271 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3181 test 0.3542 metric ['0.8769']
100%|██████████| 5/5 [02:29<00:00, 29.81s/it]


In [54]:
## Stap 2 
model = CNN_2_2().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-25 10:38:37.305 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221225-1038
100%|██████████| 938/938 [00:26<00:00, 34.89it/s]
2022-12-25 10:39:06.617 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.5227 test 0.3975 metric ['0.8577']
100%|██████████| 938/938 [00:27<00:00, 33.90it/s]
2022-12-25 10:39:36.597 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.3267 test 0.3230 metric ['0.8820']
100%|██████████| 938/938 [00:27<00:00, 34.08it/s]
2022-12-25 10:40:06.399 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.2777 test 0.2778 metric ['0.8959']
100%|██████████| 938/938 [00:27<00:00, 33.84it/s]
2022-12-25 10:40:36.383 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.2524 test 0.2807 metric ['0.8983']
100%|██████████| 938/938 [00:27<00:00, 33.78it/s]
2022-12-25 10:41:06.428 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.2225 test 0.2621 metric ['0.9039']
100%|██████████| 5/5 [02:28<00:00, 29.80s/it]


In [36]:
## Stap 2
model = CNN_2_4().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 19:16:34.429 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-1916
100%|██████████| 938/938 [00:31<00:00, 30.14it/s]
2022-12-24 19:17:07.816 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6331 test 0.5182 metric ['0.8165']
100%|██████████| 938/938 [00:33<00:00, 27.67it/s]
2022-12-24 19:17:43.810 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4611 test 0.4566 metric ['0.8389']
100%|██████████| 938/938 [00:33<00:00, 27.83it/s]
2022-12-24 19:18:19.644 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.4327 test 0.4319 metric ['0.8413']
100%|██████████| 938/938 [00:33<00:00, 27.84it/s]
2022-12-24 19:18:55.502 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.4156 test 0.4347 metric ['0.8433']
100%|██████████| 938/938 [00:33<00:00, 27.60it/s]
2022-12-24 19:19:31.606 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.4077 test 0.4553 metric ['0.8399']
100%|██████████| 5/5 [02:57<00:00, 35.41s/it]


In [41]:
## Stap 2
model = CNN_3_2().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-25 08:34:43.300 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221225-0834
100%|██████████| 938/938 [00:24<00:00, 38.65it/s]
2022-12-25 08:35:10.047 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6835 test 0.4765 metric ['0.8280']
100%|██████████| 938/938 [00:24<00:00, 38.31it/s]
2022-12-25 08:35:36.769 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4324 test 0.4040 metric ['0.8555']
100%|██████████| 938/938 [00:24<00:00, 38.78it/s]
2022-12-25 08:36:03.217 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3498 test 0.3534 metric ['0.8734']
100%|██████████| 938/938 [00:24<00:00, 38.33it/s]
2022-12-25 08:36:29.966 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3130 test 0.3481 metric ['0.8781']
100%|██████████| 938/938 [00:24<00:00, 38.67it/s]
2022-12-25 08:36:56.448 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.2950 test 0.3415 metric ['0.8810']
100%|██████████| 5/5 [02:12<00:00, 26.60s/it]


In [29]:
## Stap 2
model = CNN_3_4().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 20:23:29.482 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2023
100%|██████████| 938/938 [00:27<00:00, 34.09it/s]
2022-12-23 20:23:59.451 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6562 test 0.4600 metric ['0.8295']
100%|██████████| 938/938 [00:27<00:00, 33.90it/s]
2022-12-23 20:24:29.424 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4177 test 0.4083 metric ['0.8400']
100%|██████████| 938/938 [00:27<00:00, 34.11it/s]
2022-12-23 20:24:59.089 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3838 test 0.3833 metric ['0.8620']
100%|██████████| 938/938 [00:27<00:00, 34.25it/s]
2022-12-23 20:25:28.702 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3669 test 0.3738 metric ['0.8644']
100%|██████████| 938/938 [00:27<00:00, 34.01it/s]
2022-12-23 20:25:58.515 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3488 test 0.3868 metric ['0.8685']
100%|██████████| 5/5 [02:28<00:00, 29.77s/it]


In [42]:
## Stap 3
model = CNN_2_2_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-25 08:36:56.552 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221225-0836
100%|██████████| 938/938 [00:22<00:00, 41.91it/s]
2022-12-25 08:37:20.817 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.4818 test 0.3414 metric ['0.8790']
100%|██████████| 938/938 [00:23<00:00, 40.67it/s]
2022-12-25 08:37:45.659 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.2996 test 0.3080 metric ['0.8891']
100%|██████████| 938/938 [00:23<00:00, 40.61it/s]
2022-12-25 08:38:10.529 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.2488 test 0.2676 metric ['0.9053']
100%|██████████| 938/938 [00:23<00:00, 40.57it/s]
2022-12-25 08:38:35.404 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.2155 test 0.2705 metric ['0.9022']
100%|██████████| 938/938 [00:23<00:00, 40.61it/s]
2022-12-25 08:39:00.364 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.1921 test 0.2666 metric ['0.9048']
100%|██████████| 5/5 [02:03<00:00, 24.74s/it]


In [55]:
## Stap 3
model = CNN_2_2_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-25 10:41:06.582 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221225-1041
100%|██████████| 938/938 [00:42<00:00, 21.99it/s]
2022-12-25 10:41:52.162 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.4675 test 0.3896 metric ['0.8597']
100%|██████████| 938/938 [00:45<00:00, 20.75it/s]
2022-12-25 10:42:40.108 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.2923 test 0.2847 metric ['0.8971']
100%|██████████| 938/938 [00:45<00:00, 20.54it/s]
2022-12-25 10:43:28.484 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.2506 test 0.2868 metric ['0.8985']
100%|██████████| 938/938 [00:45<00:00, 20.62it/s]
2022-12-25 10:44:16.682 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.2157 test 0.2586 metric ['0.9075']
100%|██████████| 938/938 [00:44<00:00, 20.93it/s]
2022-12-25 10:45:04.192 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.1874 test 0.2522 metric ['0.9117']
100%|██████████| 5/5 [03:57<00:00, 47.49s/it]


In [29]:
## Stap 3
model = CNN_2_3_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 20:37:59.697 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2037
100%|██████████| 938/938 [00:25<00:00, 37.49it/s]
2022-12-23 20:38:26.583 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.5729 test 0.4638 metric ['0.8384']
100%|██████████| 938/938 [00:27<00:00, 34.09it/s]
2022-12-23 20:38:55.798 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4122 test 0.3926 metric ['0.8533']
100%|██████████| 938/938 [00:27<00:00, 34.56it/s]
2022-12-23 20:39:24.644 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3777 test 0.3874 metric ['0.8570']
100%|██████████| 938/938 [00:26<00:00, 34.97it/s]
2022-12-23 20:39:53.167 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3640 test 0.4022 metric ['0.8553']
100%|██████████| 938/938 [00:26<00:00, 34.86it/s]
2022-12-23 20:40:21.781 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3538 test 0.4140 metric ['0.8527']
100%|██████████| 5/5 [02:21<00:00, 28.39s/it]


In [35]:
## Stap 3
model = CNN_2_3_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 19:08:18.372 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-1908
100%|██████████| 938/938 [00:54<00:00, 17.19it/s]
2022-12-24 19:09:16.025 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6287 test 0.5755 metric ['0.7904']
100%|██████████| 938/938 [01:03<00:00, 14.70it/s]
2022-12-24 19:10:22.381 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4321 test 0.4249 metric ['0.8354']
100%|██████████| 938/938 [01:03<00:00, 14.78it/s]
2022-12-24 19:11:28.401 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.4120 test 0.4095 metric ['0.8540']
100%|██████████| 938/938 [01:03<00:00, 14.82it/s]
2022-12-24 19:12:34.320 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3843 test 0.4262 metric ['0.8498']
100%|██████████| 938/938 [01:03<00:00, 14.81it/s]
2022-12-24 19:13:40.226 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3723 test 0.4440 metric ['0.8360']
100%|██████████| 5/5 [05:21<00:00, 64.28s/it]


In [32]:
## Stap 3
model = CNN_2_3_64_128().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 21:03:27.623 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2103
100%|██████████| 938/938 [02:58<00:00,  5.24it/s]
2022-12-23 21:06:32.954 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.9000 test 0.4390 metric ['0.8394']
100%|██████████| 938/938 [03:35<00:00,  4.35it/s]
2022-12-23 21:10:14.508 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4217 test 0.4390 metric ['0.8444']
100%|██████████| 938/938 [03:34<00:00,  4.38it/s]
2022-12-23 21:13:54.582 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3980 test 0.4082 metric ['0.8496']
100%|██████████| 938/938 [03:33<00:00,  4.40it/s]
2022-12-23 21:17:33.435 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3900 test 0.3889 metric ['0.8567']
100%|██████████| 938/938 [03:33<00:00,  4.39it/s]
2022-12-23 21:21:12.949 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3773 test 0.4120 metric ['0.8533']
100%|██████████| 5/5 [17:45<00:00, 213.01s/it]


In [35]:
## Stap 3
model = CNN_2_4_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 19:31:23.567 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-1931
100%|██████████| 938/938 [00:24<00:00, 38.36it/s]
2022-12-24 19:31:49.957 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.5067 test 0.3925 metric ['0.8558']
100%|██████████| 938/938 [00:27<00:00, 34.52it/s]
2022-12-24 19:32:18.821 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.3526 test 0.3747 metric ['0.8678']
100%|██████████| 938/938 [00:27<00:00, 34.74it/s]
2022-12-24 19:32:47.507 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3268 test 0.3651 metric ['0.8751']
100%|██████████| 938/938 [00:27<00:00, 33.57it/s]
2022-12-24 19:33:17.162 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3070 test 0.3638 metric ['0.8759']
100%|██████████| 938/938 [00:27<00:00, 34.35it/s]
2022-12-24 19:33:46.150 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3251 test 0.4167 metric ['0.8660']
100%|██████████| 5/5 [02:22<00:00, 28.48s/it]


In [36]:
## Stap 3
model = CNN_2_4_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 19:33:46.228 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-1933
100%|██████████| 938/938 [00:58<00:00, 16.05it/s]
2022-12-24 19:34:47.453 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6504 test 0.4910 metric ['0.8196']
100%|██████████| 938/938 [01:07<00:00, 13.84it/s]
2022-12-24 19:35:57.853 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4598 test 0.4749 metric ['0.8254']
100%|██████████| 938/938 [01:07<00:00, 13.84it/s]
2022-12-24 19:37:08.306 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.4484 test 0.4241 metric ['0.8453']
100%|██████████| 938/938 [01:07<00:00, 13.92it/s]
2022-12-24 19:38:18.318 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.4151 test 0.4410 metric ['0.8334']
100%|██████████| 938/938 [01:07<00:00, 13.87it/s]
2022-12-24 19:39:28.559 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.4133 test 0.4482 metric ['0.8498']
100%|██████████| 5/5 [05:42<00:00, 68.43s/it]


In [37]:
## Stap 3
model = CNN_2_4_64_128().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 19:39:28.739 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-1939
100%|██████████| 938/938 [02:50<00:00,  5.52it/s]
2022-12-24 19:42:24.536 | INFO     | __main__:trainloop:68 - Epoch 0 train 2.4072 test 2.3035 metric ['0.0959']
100%|██████████| 938/938 [03:29<00:00,  4.47it/s]
2022-12-24 19:45:59.936 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3034 test 2.3037 metric ['0.1045']
100%|██████████| 938/938 [03:28<00:00,  4.50it/s]
2022-12-24 19:49:33.938 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3035 test 2.3043 metric ['0.1011']
100%|██████████| 938/938 [03:28<00:00,  4.49it/s]
2022-12-24 19:53:08.251 | INFO     | __main__:trainloop:68 - Epoch 3 train 2.3036 test 2.3038 metric ['0.1027']
100%|██████████| 938/938 [03:28<00:00,  4.49it/s]
2022-12-24 19:56:42.390 | INFO     | __main__:trainloop:68 - Epoch 4 train 2.3036 test 2.3039 metric ['0.1010']
100%|██████████| 5/5 [17:13<00:00, 206.67s/it]


In [ ]:
## Stap 3
model = CNN_3_2_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3
model = CNN_3_2_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3
model = CNN_3_3_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [ ]:
## Stap 3
model = CNN_3_3_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

In [31]:
## stap 3 Nieuw
model = CNN_2_2_64_128().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 20:48:10.673 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2048
100%|██████████| 938/938 [02:31<00:00,  6.21it/s]
2022-12-23 20:50:47.767 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6470 test 0.4424 metric ['0.8352']
100%|██████████| 938/938 [02:59<00:00,  5.23it/s]
2022-12-23 20:53:52.921 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4093 test 0.4006 metric ['0.8491']
100%|██████████| 938/938 [02:59<00:00,  5.21it/s]
2022-12-23 20:56:58.188 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.3823 test 0.4303 metric ['0.8507']
100%|██████████| 938/938 [02:59<00:00,  5.23it/s]
2022-12-23 21:00:03.043 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.3753 test 0.3859 metric ['0.8596']
100%|██████████| 938/938 [02:59<00:00,  5.21it/s]
2022-12-23 21:03:08.391 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3629 test 0.3986 metric ['0.8539']
100%|██████████| 5/5 [14:57<00:00, 179.49s/it]


In [29]:
## Stap 3 Nieuw
model = CNN_3_4_16_32().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 07:27:33.615 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-0727
100%|██████████| 938/938 [00:58<00:00, 16.00it/s]
2022-12-24 07:28:35.166 | INFO     | __main__:trainloop:68 - Epoch 0 train 0.6572 test 0.5043 metric ['0.8157']
100%|██████████| 938/938 [01:08<00:00, 13.69it/s]
2022-12-24 07:29:46.510 | INFO     | __main__:trainloop:68 - Epoch 1 train 0.4505 test 0.4547 metric ['0.8222']
100%|██████████| 938/938 [01:08<00:00, 13.72it/s]
2022-12-24 07:30:57.559 | INFO     | __main__:trainloop:68 - Epoch 2 train 0.4240 test 0.4333 metric ['0.8523']
100%|██████████| 938/938 [01:07<00:00, 13.81it/s]
2022-12-24 07:32:08.244 | INFO     | __main__:trainloop:68 - Epoch 3 train 0.4075 test 0.4399 metric ['0.8529']
100%|██████████| 938/938 [01:08<00:00, 13.75it/s]
2022-12-24 07:33:19.177 | INFO     | __main__:trainloop:68 - Epoch 4 train 0.3965 test 0.5124 metric ['0.8167']
100%|██████████| 5/5 [05:45<00:00, 69.08s/it]


In [ ]:
## Stap 3 Nieuw
model = CNN_3_4_32_64().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-23 21:21:55.969 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221223-2121
100%|██████████| 938/938 [00:58<00:00, 15.99it/s]
2022-12-23 21:22:57.759 | INFO     | __main__:trainloop:68 - Epoch 0 train 2.3266 test 2.3040 metric ['0.1028']
100%|██████████| 938/938 [01:07<00:00, 13.97it/s]
2022-12-23 21:24:07.827 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3037 test 2.3033 metric ['0.1024']
100%|██████████| 938/938 [01:07<00:00, 13.99it/s]
2022-12-23 21:25:17.753 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3032 test 2.3031 metric ['0.1032']
100%|██████████| 938/938 [01:06<00:00, 14.17it/s]
2022-12-23 21:26:26.941 | INFO     | __main__:trainloop:68 - Epoch 3 train 2.3034 test 2.3043 metric ['0.1001']
100%|██████████| 938/938 [01:06<00:00, 14.16it/s]
2022-12-23 21:27:36.028 | INFO     | __main__:trainloop:68 - Epoch 4 train 2.3038 test 2.3032 metric ['0.1016']
100%|██████████| 5/5 [05:39<00:00, 67.98s/it]


In [28]:
## Stap 3 Nieuw
model = CNN_3_4_64_128().to(device)

model = trainloop(
                model=model,
                metrics=[accuracy],
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                train_steps=len(train_dataloader),
                eval_steps=150,
                )

2022-12-24 07:38:31.702 | INFO     | __main__:dir_add_timestamp:39 - Logging to /home/admindme/code/ML22_opdracht1/models/test/20221224-0738
100%|██████████| 938/938 [02:50<00:00,  5.49it/s]
2022-12-24 07:41:28.430 | INFO     | __main__:trainloop:68 - Epoch 0 train 2.4089 test 2.3039 metric ['0.0939']
100%|██████████| 938/938 [03:27<00:00,  4.52it/s]
2022-12-24 07:45:01.406 | INFO     | __main__:trainloop:68 - Epoch 1 train 2.3035 test 2.3034 metric ['0.0940']
100%|██████████| 938/938 [03:27<00:00,  4.52it/s]
2022-12-24 07:48:34.520 | INFO     | __main__:trainloop:68 - Epoch 2 train 2.3034 test 2.3038 metric ['0.0933']
100%|██████████| 938/938 [03:28<00:00,  4.51it/s]
2022-12-24 07:52:08.143 | INFO     | __main__:trainloop:68 - Epoch 3 train 2.3033 test 2.3047 metric ['0.1014']
100%|██████████| 938/938 [03:26<00:00,  4.54it/s]
2022-12-24 07:55:40.430 | INFO     | __main__:trainloop:68 - Epoch 4 train 2.3036 test 2.3030 metric ['0.1014']
100%|██████████| 5/5 [17:08<00:00, 205.67s/it]
